In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
import os
warnings.filterwarnings('ignore')
os.chdir("../../")

In [3]:
import time

In [4]:
from juggling_dlx_milp import *

In [5]:
import modele
import ipywidgets as ipw
import pythreejs

colors = ["blue", "red", "green", "yellow", "purple", "cyan", "magenta"]
sides = [-1, 1, 1]

# Test GUI

In [6]:
import logging
logger = logging.getLogger()

In [7]:
out = ipw.Output()

l_inttext = ipw.Layout(width='100px')
l_text = ipw.Layout(width='150px')
l_simulation_text = ipw.Layout(width='400px')

w_nb_hands = ipw.IntText(
    value=2,
    disabled=False,
    layout=l_inttext
)
w_max_weight = ipw.IntText(
    value=3,
    disabled=False,
    layout=l_inttext
)
w_max_height = ipw.IntText(
    value=5,
    disabled=False,
    layout=l_inttext
)
w_no_multiplex = ipw.Checkbox(
    value=False,
    disabled=False,
    indent=False
)
w_generate_forbidden_multiplex = ipw.Button(
    description='Générer',
    disabled=False,
    button_style='',
    icon='refresh'
)
w_forbidden_multiplex = ipw.SelectMultiple(
    options=[''],
    value=[],
    disabled=False,   
)
def fill_forbidden_multiplex(args):
    global w_forbidden_multiplex
    old_options = w_forbidden_multiplex.options
    selected = w_forbidden_multiplex.value
    max_height = w_max_height.value
    options = []
    for i in range(1, max_height + 1):
        for j in range(i, max_height + 1):
            options.append(str(i) + ", " + str(j))
    w_forbidden_multiplex.options = options
    w_forbidden_multiplex.disabled = w_no_multiplex.value
    new_selected = []
    for s in selected:
        i = int(s.split(', ')[0])
        j = int(s.split(', ')[1])
        if i <= max_height and j <= max_height:
            new_selected.append(str(i) + ', ' + str(j))
    w_forbidden_multiplex.value = new_selected
    logger.debug(old_options)
    if len(old_options) == 1:
        w_forbidden_multiplex.selected = ["1, 1", "1, 2", "1, 3", "1, 4", "1, 5"]
    out.clear_output()
    with out:
        display(ui)
w_maximize = ipw.Text(
    value='',
    placeholder='',
    disabled=False
)
w_generate_hands = ipw.Button(
    description='Générer',
    disabled=False,
    button_style='',
    icon='refresh'
)
hands_constraints = {}
w_hands_constraints = ipw.Accordion([])
def fill_hand_constraints(args):
    global hands_constraints, w_hands_constraints
    max_weight_old = len(hands_constraints) + 1
    max_weight = w_max_weight.value
    if max_weight > max_weight_old:
        for k in range(max_weight_old + 1, max_weight + 1):
            w_throw = ipw.BoundedIntText(value=1, min=1, max=k, step=1, disabled=False, layout=l_text)
            w_catch = ipw.BoundedIntText(value=k, min=1, max=k, step=1, disabled=False, layout=l_text)
            w_perm = ipw.Text(value='', placeholder='', disabled=False, layout=l_text)
            hands_constraints[k] = {
                'throw': w_throw,
                'catch': w_catch,
                'perm': w_perm,
                'box': ipw.GridBox([
                    ipw.Label('Lancer :'), w_throw,
                    ipw.Label('Récupération :'), w_catch,
                    ipw.Label('Permutations :'), w_perm
                ], layout=ipw.Layout(grid_template_columns="repeat(2, 100px)"))
            }
    else:
        for k in range(max_weight + 1, max_weight_old + 1):
            hands_constraints.pop(k)
    w_hands_constraints = ipw.Accordion([hands_constraints[k]['box'] for k in range(2, max_weight + 1)], layout=ipw.Layout(width='300px'))
    for k in range(2, max_weight + 1):
        w_hands_constraints.set_title(index=k - 2, title=str(k) + " mains")
    tab.children = [tab1, tab2, tab3()]
    out.clear_output()
    with out:
        display(ui)
def tab3():
    return ipw.GridBox([
        ipw.Label('Contraintes sur les mains :'), w_generate_hands,
        ipw.Label(''), w_hands_constraints
    ], layout=ipw.Layout(grid_template_columns="repeat(2, 210px)"))
tab1 = ipw.GridBox([
    ipw.Label('Nombre de mains :'), w_nb_hands,
    ipw.Label('Nombre maximal de balles dans une main :'), w_max_weight,
    ipw.Label('Hauteur maximale :'), w_max_height
], layout=ipw.Layout(grid_template_columns="repeat(2, 260px)"))
tab2 = ipw.GridBox([
    ipw.Label('Maximiser les lancers :'), w_maximize,
    ipw.Label('Aucun lancer multiple :'), w_no_multiplex,
    ipw.Label('Lancers multiples interdits :'), w_generate_forbidden_multiplex,
    ipw.VBox([
        ipw.Label('(maintenir Ctrl pour', layout=ipw.Layout(height='20px')),
        ipw.Label('sélectionner/désélectionner)', layout=ipw.Layout(height='20px'))]), 
    w_forbidden_multiplex
], layout=ipw.Layout(grid_template_columns="repeat(2, 170px)"))

tab = ipw.Tab(layout=ipw.Layout(width='550px', margin='0px 0px 10px 0px'))
tab.children = [tab1, tab2, tab3()]
tab.set_title(index=0, title="Général")
tab.set_title(index=1, title="Lancers")
tab.set_title(index=2, title="Mains")

model = modele.Model({}, [[], []])
play = ipywidgets.Play(
    value=0,
    min=0,
    max=4000,
    step=5,
    interval=30,
    description="Press play",
    disabled=False
)
slider = ipywidgets.IntSlider(min=0, max=4000)
ipywidgets.jslink((play, 'value'), (slider, 'value'))
view = modele.View(model, sides)
slider.observe(lambda change: view.update(change['new'] / 100, change['old'] / 100), names="value")

w_music = ipw.Textarea(
    value='[1 do] [2 do] [3 do] [4 re] [5 mi] [7 re] [9 do] [10 mi] [11 re] [12 re] [13 do]',
    placeholder='',
    description='',
    disabled=False,
    layout=ipw.Layout(width='475px', margin='0px 0px 10px 0px')
)
w_working = ipw.Label('Prêt')
w_solve = ipw.Button(
    description='Résoudre les contraintes',
    disabled=False,
    button_style='',
    icon='check',
    layout=ipw.Layout(width='345px')
)
w_simulate = ipw.Button(
    description='Simuler',
    disabled=True,
    button_style='',
    icon='check',
    layout=ipw.Layout(width='200px')
)
w_step = ipw.IntText(
    value=5,
    disabled=False,
    layout=l_inttext
)
w_sides = ipw.Text(
    value='-1, 1',
    placeholder='',
    disabled=False,
    layout=l_simulation_text
)
w_colors = ipw.Text(
    value='blue, red, green, yellow, purple, cyan, magenta',
    placeholder='',
    disabled=False,
    layout=l_simulation_text
)
w_method = ipw.RadioButtons(
    options=['Programmation Linéaire (plus rapide, ne respecte pas les contraintes sur les mains)',
             'Dancing Links (plus lent, respecte les contraintes sur les mains)'],
    description='',
    disabled=False,
    layout={'width': 'max-content'}
)

# balls = {}
# pattern = [[], []]
jsol = None

def ui_view(view, play, slider):
    return ipw.HBox([
        ipw.VBox([
            ipw.HBox([ipw.Label('Musique :', layout=ipw.Layout(width='70px')), w_music]),
            tab, 
            ipw.GridBox([
                ipw.Label('Pas :'), w_step,
                ipw.Label('Orientation des mains :'), w_sides,
                ipw.Label('Couleurs :'), w_colors
            ], layout=ipw.Layout(grid_template_columns='repeat(2, 150px)')),
            w_working,
            w_method,
            ipw.HBox([w_solve, w_simulate])], layout=ipw.Layout(margin="10px")), 
        ipywidgets.VBox([view.widget, ipywidgets.HBox([play, slider])], layout=ipw.Layout(margin="10px"))
    ])

def solve(args):
    global jsol
    
    if w_working.value != "Prêt":
        return
    
    music = []
    for s in w_music.value.split('] ['):
        s1 = s.strip()
        if s1[0] == '[':
            s1 = s1[1:]
        elif s1[-1] == ']':
            s1 = s1[:-1]
        p = s1.split(' ')
        for i in range(1, len(p)):
            music.append((int(p[0]), p[i]))

    nb_hands = w_nb_hands.value
    max_height = w_max_height.value
    max_weight = w_max_weight.value
    forbidden_multiplex = []
    for s in w_forbidden_multiplex.value:
        i = int(s.split(', ')[0])
        j = int(s.split(', ')[1])
        forbidden_multiplex.append((i, j))
    maximize = [int(s) for s in w_maximize.value.split(' ')]
    method = 'DLX' if w_method.value.startswith('Dancing Links') else 'MILP'
    optimize = w_maximize.value != ""
    
    w_working.value = "En cours..."
    
    balls, throws = music_to_throws(music)
    
    ec_instance = throws_to_extended_exact_cover(balls, throws, nb_hands, max_height, max_weight,
                                                 forbidden_multiplex, True)
    sol = None
    if method == "DLX":
        sol = get_solution_with_dlx(ec_instance, maximize=maximize)
    elif method == "MILP":
        sol = solve_exact_cover_with_milp(ec_instance, optimize=optimize, maximize=maximize)
    if len(sol) == 0:
        raise RuntimeError("No solution.")
    jsol = exact_cover_solution_to_juggling_solution(sol)
    
    w_working.value = "Prêt"
    w_simulate.disabled = False

def simulate(args):
    colors = w_colors.value.split(', ')
    sides = [int(x) for x in w_sides.value.split(', ')]
    step = w_step.value
    
    balls, pattern = juggling_sol_to_simulator(jsol, colors)
    model = modele.Model(balls, pattern)
    play = ipywidgets.Play(
        value=0,
        min=0,
        max=4000,
        step=step,
        interval=30,
        description="Press play",
        disabled=False
    )
    slider = ipywidgets.IntSlider(min=0, max=4000)
    ipywidgets.jslink((play, 'value'), (slider, 'value'))
    view = modele.View(model, sides, out)
    slider.observe(lambda change: view.update(change['new'] / 100, change['old'] / 100), names="value")
    with out:
        out.clear_output()
        display(ui_view(view, play, slider))
    

w_generate_forbidden_multiplex.on_click(fill_forbidden_multiplex)
w_generate_hands.on_click(fill_hand_constraints)
w_solve.on_click(solve)
w_simulate.on_click(simulate)

ui = ui_view(view, play, slider)

with out:
    display(ui)
out

Output()

- [x] widget couleurs
- [x] widget music
- [x] widget sides
- [x] widget selection de la méthode
- [x] lire le widget maximize et paramètrer la méthode MILP
- [x] corriger le codage des multiplex interdits
- [x] widget step
- [ ] mécanisme de sauvegarde d'une séquence trouvée
- [ ] interdiction d'une hauteur de lancer
- [ ] lancers interdits par initialement

# Musique à jouer

In [8]:
# Au clair de la lune
music = [( 1, "do"), ( 2, "do"), ( 3, "do"), 
         ( 4, "re"), ( 5, "mi"), ( 7, "re"), 
         ( 9, "do"), (10, "mi"), (11, "re"),
         (12, "re"), (13, "do")]

In [10]:
music = [(1, "re5"), (2, "do5"), (3, "re5"), (4, "do5"),
         (5, "re5"), (7, "do5"), (9, "mi5"),
         (11, "re5"), (12, "fa5"), (13, "la5"), (14, "re6"), (15, "fa6")]

In [11]:
# Frères jacques
music = [( 1, "fa4"), ( 2, "sol4"), ( 3, "la4"), ( 4, "fa4"),
         ( 5, "fa4"), ( 6, "sol4"), ( 7, "la4"), ( 8, "fa4"),
         ( 9, "la4"), (10, "la#4"), (11, "do5"),
         (13, "la4"), (14, "la#4"), (15, "do5")]

In [12]:
# In the hall of the moutain king
music = [( 1, "la3"), ( 2, "si3"), ( 3, "do4"), ( 4, "re4"),
         ( 5, "mi4"), ( 6, "do4"), ( 7, "mi4"),
         ( 9, "re#4"), (10, "si3"), (11, "re#4"),
         (13, "re4"), (14, "la#3"), (15, "re4")]

In [13]:
# Tetris
music = [(1, "mi5"), 
         (3, "si4"), (4, "do5"), (5, "re5"),
         (7, "do5"), (8, "si4"), (9, "la4"), 
         (11, "la4"), (12, "do5"), (13, "mi5"),
         (15, "re5"), (16, "do5"), (17, "si4"),
         (20, "do5"), (21, "re5"), (23, "mi5"),
         (25, "do5"), (27, "la4"), (29, "la4")]

# Contraintes

In [13]:
nb_hands = 2
max_height = 5
max_weight = 3
# forbidden_multiplex = [(1, 2), (1, 3), (1, 4), (2, )]
forbidden_multiplex = [(1, 1)]
multiple_throws = True

# Algorithme

In [14]:
balls, throws = music_to_throws(music)

In [21]:
ec_instance = throws_to_extended_exact_cover(balls, throws, nb_hands, max_height, max_weight,
                                             forbidden_multiplex, multiple_throws)

In [22]:
# ec_instance.rows

In [23]:
# voir si possibilité de redéfinition
sol = solve_and_print(music, nb_hands, max_height, max_weight, forbidden_multiplex, method="MILP", maximize=[3, 4])

  {}   {'mi', 'do', 're'} : do -- 1 --> 0
                            re -- 4 --> 1
{'do'}       {'mi'}       : mi -- 4 --> 1
                            do -- 1 --> 1
  {}         {'do'}       : do -- 1 --> 0
{'do'}         {}         : 
{'do'}       {'re'}       : re -- 3 --> 0
{'do'}       {'mi'}       : 
{'do'}       {'mi'}       : do -- 3 --> 0
                            mi -- 4 --> 1
{'re'}         {}         : re -- 4 --> 0
  {}           {}         : 
{'do'}         {}         : 
{'do'}       {'mi'}       : do -- 3 --> 1
{'re'}       {'mi'}       : re -- 1 --> 1
  {}      {'mi', 're'}    : 


In [18]:
solve_and_simulate(music, nb_hands, max_height, max_weight, forbidden_multiplex, colors, sides, method="DLX", step=5)

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

## Résolution avec Exact Cover

In [9]:
sol1 = get_solution_with_dlx(ec_instance)

In [10]:
len(sol1.rows)

11

In [11]:
print_juggling(sol1)

{'re', 'do', 'mi'}         {}         : do -- 1 --> 1
   {'re', 'mi'}          {'do'}       : do -- 1 --> 0
{'re', 'do', 'mi'}         {}         : do -- 1 --> 1
   {'re', 'mi'}          {'do'}       : re -- 1 --> 1
      {'mi'}          {'re', 'do'}    : mi -- 1 --> 1
        {}         {'re', 'do', 'mi'} : 
        {}         {'re', 'do', 'mi'} : re -- 1 --> 0
      {'re'}          {'do', 'mi'}    : 
      {'re'}          {'do', 'mi'}    : do -- 1 --> 0
   {'re', 'do'}          {'mi'}       : mi -- 1 --> ?
   {'re', 'do'}            {}         : re -- 1 --> 1
      {'do'}             {'re'}       : re -- 1 --> ?
      {'do'}               {}         : do -- 1 --> ?


In [12]:
balls1, pattern1 = juggling_sol_to_simulator(sol1, colors)

sides1 = [-1, 1, 1]

model1 = modele.Model(balls1, pattern1)
slider1 = ipywidgets.FloatSlider(min=0, max=40, step=0.05)
view1 = modele.View(model1, sides1)
slider1.observe(lambda change: view1.update(change['new'], change['old']), names="value")
ipywidgets.VBox([view1.widget,slider1])

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

## Résolution avec MILP

In [9]:
sol2 = solve_exact_cover_with_milp(ec_instance, True)

In [10]:
len(sol2.rows)

11

In [11]:
print_juggling(sol2)

{'mi', 're', 'do'}      {}      : do -- 1 --> 1
   {'mi', 're'}       {'do'}    : re -- 3 --> 1
                                  mi -- 4 --> 1
                                  do -- 1 --> 0
      {'do'}            {}      : do -- 1 --> 1
        {}            {'do'}    : 
        {}         {'re', 'do'} : re -- 3 --> 0
        {}         {'mi', 'do'} : 
        {}         {'mi', 'do'} : do -- 3 --> 0
      {'re'}          {'mi'}    : mi -- 3 --> ?
      {'re'}            {}      : re -- 3 --> 1
      {'do'}            {}      : 
      {'do'}            {}      : do -- 3 --> ?
        {}            {'re'}    : re -- 1 --> ?
        {}              {}      : 


In [12]:
balls2, pattern2 = juggling_sol_to_simulator(sol2, colors)

sides2 = [-1, 1, 1]

model2 = modele.Model(balls2, pattern2)
slider2 = ipywidgets.FloatSlider(min=0, max=40, step=0.05)
view2 = modele.View(model2, sides2)
slider2.observe(lambda change: view2.update(change['new'], change['old']), names="value")
ipywidgets.VBox([view2.widget,slider2])

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

## Fonctions tout en un

In [7]:
solve_and_simulate(music, nb_hands, max_height, max_weight, forbidden_multiplex, colors, sides, method="DLX")

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

In [8]:
solve_and_simulate(music, nb_hands, max_height, max_weight, forbidden_multiplex, colors, sides, method="MILP")

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

In [19]:
solve_and_print(music, nb_hands, max_height, max_weight, forbidden_multiplex, method="DLX")

{'re', 'do', 'mi'}         {}         : do -- 1 --> 1
   {'re', 'mi'}          {'do'}       : do -- 1 --> 0
{'re', 'do', 'mi'}         {}         : do -- 1 --> 1
   {'re', 'mi'}          {'do'}       : re -- 1 --> 1
      {'mi'}          {'re', 'do'}    : mi -- 1 --> 1
        {}         {'re', 'do', 'mi'} : 
        {}         {'re', 'do', 'mi'} : re -- 1 --> 0
      {'re'}          {'do', 'mi'}    : 
      {'re'}          {'do', 'mi'}    : do -- 1 --> 0
   {'re', 'do'}          {'mi'}       : mi -- 1 --> ?
   {'re', 'do'}            {}         : re -- 1 --> 1
      {'do'}             {'re'}       : re -- 1 --> ?
      {'do'}               {}         : do -- 1 --> ?


In [20]:
solve_and_print(music, nb_hands, max_height, max_weight, forbidden_multiplex, method="MILP")

{'re', 'do', 'mi'}      {}      : do -- 1 --> 1
   {'re', 'mi'}       {'do'}    : re -- 3 --> 1
                                  mi -- 4 --> 1
                                  do -- 1 --> 0
      {'do'}            {}      : do -- 1 --> 1
        {}            {'do'}    : 
        {}         {'re', 'do'} : re -- 3 --> 0
        {}         {'do', 'mi'} : 
        {}         {'do', 'mi'} : do -- 3 --> 0
      {'re'}          {'mi'}    : mi -- 3 --> ?
      {'re'}            {}      : re -- 3 --> 1
      {'do'}            {}      : 
      {'do'}            {}      : do -- 3 --> ?
        {}            {'re'}    : re -- 1 --> ?
        {}              {}      : 
